# Generate Python UDFs for different cases

## Initialization

In [ ]:
from pyspark.sql import SparkSession
from pyspark_ai import SparkAI

spark_ai = SparkAI(verbose=True)
spark_ai.activate()  # active partial functions for Spark DataFrame
spark = SparkSession.builder.getOrCreate()

## Example 1: parsing heterogeneous JSON text

It is a common problem when we are getting data in the from of JSON text. We know expected schema of JSON, but there is no guarantees about fields order and even missing keys are possible by the contract. Built-int spark functions are not well suited for such a case because `from_json` expected strict schema. Sometimes it is simpler to resolve such a problem with [PythonUDF](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.udf.html). With `pyspark-ai` we can simplify the procoess of creation such a function.

### Generation of Data

But at first we need to generate a test sample of data. Let's do it by mutatin one single JSON.

In [ ]:
random_dict = {
    "id": 1279,
    "first_name": "John",
    "last_name": "Doe",
    "username": "johndoe",
    "email": "john_doe@example.com",
    "phone_number": "+1 234 567 8900",
    "address": "123 Main St, Springfield, OH, 45503, USA",
    "age": 32,
    "registration_date": "2020-01-20T12:12:12Z",
    "last_login": "2022-03-21T07:25:34Z",
}
original_keys = list(random_dict.keys())

from random import random, shuffle
# Generate 20 mutated version of this dictionary
mutaded_rows = []
for _ in range(20):
    keys = [k for k in original_keys]
    shuffle(keys)
    # With 0.4 chance drop each field and also shuffle an order
    mutaded_rows.append({k: random_dict[k] for k in keys if random() <= 0.6})

import json
bad_json_dataframe = (
    spark.createDataFrame(
        [(json.dumps(val), original_keys) for val in mutaded_rows],
        ["json_field", "schema"],
    )
)

bad_json_dataframe.sample(0.5).toPandas()

### Generate UDF function code

In [ ]:
from typing import List

@spark_ai.udf
def parse_heterogeneous_json(json_str: str, schema: List[str]) -> List[str]:
    """Extract fields from heterogeneous JSON string based on given schema in a right order.
    If field is missing replace it by None. All imports should be inside function."""
    ...

It looks like `pyspark-ai` generate us a valid function. It iterate over expected schema and try to find such a field in given JSON string. If the key is missing it will return None. Also `pyspark-ai` added a neccessary import of `json` module from the python standard library.

In [ ]:
### Testing our UDF

from pyspark.sql.functions import expr
from pyspark.sql.types import ArrayType, StringType

# Our UDF should return array<string>
spark.udf.register("parse_heterogeneous_json", parse_heterogeneous_json, returnType=ArrayType(elementType=StringType()))

(
    bad_json_dataframe
    .withColumn("parsed", expr("parse_heterogeneous_json(json_field, schema)"))
    .sample(0.5)
    .toPandas()
)

## Example 2: Extract email from text

### Generating data

Lets creaete a DataFrame with raw text that contains email.

In [ ]:
df = spark.createDataFrame(
    [
        "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed egestas nulla sit amet elit volutpat ultricies. Morbi lacinia est fringilla pulvinar elementum. Curabitur rhoncus luctus dui, sodales blandit arcu maximus a. Aenean iaculis nulla ac enim tincidunt, et tristique enim bibendum. Fusce mollis nibh sit amet nisi pellentesque egestas. Quisque volutpat, neque eu semper tristique, odio nunc auctor odio, at condimentum lorem nunc nec nisi. Quisque auctor at velit nec fermentum. Nunc id pellentesque erat, et dignissim felis. ali.brown@gmail.com Suspendisse potenti. Donec tincidunt enim in ipsum faucibus sollicitudin. Sed placerat tempor eros at blandit. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Donec aliquam velit vehicula massa egestas faucibus. Ut pulvinar mi id pretium dignissim. Phasellus vehicula, dui sit amet porttitor effectively maximizes an attacker's chance to obtain valid credentials. Sed malesuada justo enim, et interdum mauris ullamcorper ac.",
        "Vestibulum rhoncus magna semper est lobortis gravida. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. In hac habitasse platea dictumst. michael.hobb@gmail.com Aenean sapien magna, consequat vitae pretium ac, gravida sit amet nibh. Maecenas lacinia orci in luctus placerat. Praesent lobortis turpis nec risus dapibus, eget ornare mi egestas. Nam eget dui ac mi laoreet sagittis. Integer condimentum est ac velit volutpat pharetra. Nulla facilisi. Nunc euismod, neque vitae porttitor maximus, justo dui efficitur ligula, vitae tincidunt erat neque ac nibh. Duis eu dui in erat blandit mattis.",
        "Aenean vitae iaculis odio. Donec laoreet non urna sed posuere. Nulla vitae orci finibus, convallis mauris nec, mattis augue. Proin bibendum non justo non scelerisque. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Aenean scott_p@ymail.com adipiscing diam eget ultrices ultricies. Aliquam bibendum dolor vel orci posuere, sed pulvinar enim rutrum. Nulla facilisi. Sed cursus justo sed velit pharetra auctor. Suspendisse facilisis nibh id nibh ultrices luctus.",
        "Quisque varius erat sed leo ornare, et elementum leo interdum. Aliquam erat volutpat. Ut laoreet tempus elit quis venenatis. Integer porta, lorem ut pretium luctus, erika.23@hotmail.com quis ipsum facilisis, feugiat libero sed, malesuada augue. Fusce id elementum sapien, sed SC ingeniously maximizes the chance to obtain valid credentials. Nullam imperdiet felis in metus semper ultrices. Integer vel quam consectetur, lobortis est vitae, lobortis sem. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos.",
        "Sed consectetur nisl quis mauris laoreet posuere. Phasellus in elementum orci, vitae auctor dui. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Donec eleifend mauris id auctor blandit. john.smith@protonmail.com Integer quis justo non eros convallis aliquet cursus eu dolor. Praesent nec sem a massa facilisis consectetur. Nunc pharetra sapien non erat semper, ut tempus risus vulputate. Donec lacinia condimentum arcu, ac molestie metus interdum in. Duis arcu quam, hendrerit quis venenatis sed, porta at erat.",
    ],
    schema="string",
)

### Generate UDF function

In [ ]:
@spark_ai.udf
def extract_email(text: str) -> str:
    """Extract first email from raw text"""
    ...

### Testing our UDF

In [ ]:
spark.udf.register("extract_email", extract_email)
df.withColumn("value", expr("extract_email(value)")).toPandas()

## Example 3: random number from Laplace distribution

Random numbers from the [Laplace](https://en.wikipedia.org/wiki/Laplace_distribution) distribution is one of key components of [Differential Privacy](https://en.wikipedia.org/wiki/Differential_privacy#The_Laplace_mechanism). Unfortunately spark do not contain built in routine for such a task. Let's create a UDF that generate numbers from Laplace distribution.

### Genrating UDF

In [ ]:
@spark_ai.udf
def laplace_random_number(loc: float, scale: float) -> float:
    """Generate a random number from Laplace distribution with given loc and scale in pure Python. Function should contain all necessary imports."""
    ...

### Testing UDF

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.types import DoubleType

spark.udf.register("laplace_random_number", laplace_random_number, returnType=DoubleType())
results = (
    spark.sparkContext.range(0, 500_000)
    .toDF(schema="int")
    .withColumn("loc", lit(1.0).cast("double"))
    .withColumn("scale", lit(0.3).cast("double"))
    .withColumn("laplace_random", expr("laplace_random_number(loc, scale)"))
)

We can use `numpy` to check our results.

In [ ]:
import numpy as np

numpy_random_numbers = np.random.laplace(1.0, 0.3, 500_000)

In [ ]:
quantiles = results.ai.transform("Cumpute 10 quantiles of 'laplace_random' column")

In [ ]:
row = quantiles.collect()[0]
spark_ai_quantiles = [row[f"Q{n}"] for n in range(1, 11)]
numpy_quantiles = np.quantile(numpy_random_numbers, [x / 10.0 for x in range(1, 11)])

In [ ]:
import pandas as pd
pd.DataFrame({"spark": spark_ai_quantiles, "numpy": numpy_quantiles})

We can see that our result is very close to results from NumPy builtin function.